In [3]:
import pandas as pd
import numpy as np
import os

# Set working directory and define paths
base_dir = "C:/Users/uabic/Desktop/pAML_1"
data_file = os.path.join(base_dir, "merged_expr_AML.csv")
normal_dir = os.path.join(base_dir, "pAML_1", "Normal")
tumor_dir = os.path.join(base_dir, "pAML_1", "Tumor")

os.makedirs(normal_dir, exist_ok=True)
os.makedirs(tumor_dir, exist_ok=True)

import pandas as pd
import numpy as np
import os

# Define sample information
sample_info = {
    "GSM187698.CEL.gz": "T", "GSM187701.CEL.gz": "T", "GSM187704.CEL.gz": "T", "GSM187707.CEL.gz": "T",
    "GSM187710.CEL.gz": "T", "GSM187713.CEL.gz": "T", "GSM187716.CEL.gz": "T", "GSM187719.CEL.gz": "T",
    "GSM187722.CEL.gz": "T", "GSM187725.CEL.gz": "T", "GSM187728.CEL.gz": "T", "GSM187731.CEL.gz": "T",
    "GSM187734.CEL.gz": "T", "GSM187737.CEL.gz": "T", "GSM187740.CEL.gz": "T", "GSM187743.CEL.gz": "T",
    "GSM187746.CEL.gz": "T", "GSM187749.CEL.gz": "T", "GSM187752.CEL.gz": "T", "GSM187755.CEL.gz": "T",
    "GSM187758.CEL.gz": "T", "GSM187761.CEL.gz": "T", "GSM187764.CEL.gz": "T", "GSM187767.CEL.gz": "T",
    "GSM187770.CEL.gz": "T", "GSM187771.CEL.gz": "T", "GSM187772.CEL.gz": "T", "GSM187780.CEL.gz": "T",
    "GSM187781.CEL.gz": "T", "GSM187782.CEL.gz": "T", "GSM187788.CEL.gz": "T", "GSM187789.CEL.gz": "T",
    "GSM187790.CEL.gz": "T",
    "GSM100895.CEL.gz": "N", "GSM100896.CEL.gz": "N", "GSM100897.CEL.gz": "N", "GSM100898.CEL.gz": "N",
    "GSM100901.CEL.gz": "N"
}

# Read the merged expression data
merged_data = pd.read_csv(data_file)

# Assuming the first column is 'Gene' and the rest are sample columns
genes = merged_data.iloc[:, 0]
data = merged_data.iloc[:, 1:]

# Extract sample IDs from column names and filter based on sample_info
tumor_samples = [col for col in data.columns if sample_info.get(col.split(".")[0] + ".CEL.gz") == "T"]
normal_samples = [col for col in data.columns if sample_info.get(col.split(".")[0] + ".CEL.gz") == "N"]

tumor_data = data[tumor_samples]
normal_data = data[normal_samples]

# Add the gene names back as the first column
tumor_data.insert(0, 'Gene', genes)
normal_data.insert(0, 'Gene', genes)

# Remove duplicate gene name rows
tumor_data = tumor_data[~tumor_data['Gene'].duplicated(keep='first')]
normal_data = normal_data[~normal_data['Gene'].duplicated(keep='first')]

# Log normalize the data
tumor_data.iloc[:, 1:] = np.log1p(tumor_data.iloc[:, 1:])
normal_data.iloc[:, 1:] = np.log1p(normal_data.iloc[:, 1:])

# Save the processed data
tumor_data.to_csv(os.path.join(tumor_dir, "processed_tumor_data2.csv"), index=False)
normal_data.to_csv(os.path.join(normal_dir, "processed_normal_data2.csv"), index=False)

print("Data processed and saved successfully.")



Data processed and saved successfully.


In [4]:
import numpy as np
from scipy.stats import spearmanr

# Function to create adjacency matrix using Spearman correlation
def create_adjacency_matrix(data):
    gene_names = data.iloc[:, 0]
    expression_data = data.iloc[:, 1:].values
    corr_matrix = spearmanr(expression_data, axis=1).correlation
    np.fill_diagonal(corr_matrix, 0)  # Remove self-correlations
    return pd.DataFrame(corr_matrix, index=gene_names, columns=gene_names)

# Read the processed data
processed_tumor_data = pd.read_csv(os.path.join(tumor_dir, "processed_tumor_data2.csv"))
processed_normal_data = pd.read_csv(os.path.join(normal_dir, "processed_normal_data2.csv"))

# Create adjacency matrices
tumor_adj_matrix = create_adjacency_matrix(processed_tumor_data)
normal_adj_matrix = create_adjacency_matrix(processed_normal_data)

# Save adjacency matrices
tumor_adj_matrix.to_csv(os.path.join(tumor_dir, "tumor_adjacency_matrix2.csv"))
normal_adj_matrix.to_csv(os.path.join(normal_dir, "normal_adjacency_matrix2.csv"))

print("Adjacency matrices created and saved successfully.")


Adjacency matrices created and saved successfully.


In [1]:
# Define directories
base_dir = "C:/Users/uabic/Desktop/pAML_1"
normal_dir = os.path.join(base_dir, "pAML_1", "Normal")
tumor_dir = os.path.join(base_dir, "pAML_1", "Tumor")

import pandas as pd
import numpy as np
from pybdm import BDM
from pybdm import PerturbationExperiment
import matplotlib.pyplot as plt
import os


# Load adjacency matrices
tumor_adj_matrix = pd.read_csv(os.path.join(tumor_dir, "tumor_adjacency_matrix2.csv"), index_col=0)
normal_adj_matrix = pd.read_csv(os.path.join(normal_dir, "normal_adjacency_matrix2.csv"), index_col=0)

# Perform BDM perturbation analysis
def binarize_matrix(matrix, threshold=0.5):
    binary_matrix = (matrix > threshold).astype(int)
    return binary_matrix

def bdm_perturbation_analysis(adj_matrix):
    bdm = BDM(ndim=2)
    binary_matrix = binarize_matrix(adj_matrix.values)
    perturbation = PerturbationExperiment(bdm, binary_matrix, metric='bdm')
    delta_bdm = perturbation.run()
    
    # Ensure the shape matches the original matrix
    reshaped_delta_bdm = np.reshape(delta_bdm, adj_matrix.shape)
    
    return pd.DataFrame(reshaped_delta_bdm, index=adj_matrix.index, columns=adj_matrix.columns)

tumor_bdm_results = bdm_perturbation_analysis(tumor_adj_matrix)
normal_bdm_results = bdm_perturbation_analysis(normal_adj_matrix)

# Save BDM results
tumor_bdm_results.to_csv(os.path.join(tumor_dir, "tumor_bdm_results2.csv"))
normal_bdm_results.to_csv(os.path.join(normal_dir, "normal_bdm_results2.csv"))

# Plot top 10 BDM perturbations
def plot_bdm_results(bdm_results, title, save_path):
    top_10 = bdm_results.unstack().nlargest(10)
    plt.figure(figsize=(10, 6))
    plt.bar(top_10.index.map(str), top_10.values)
    plt.xlabel('Gene Pair')
    plt.ylabel('Perturbation Score')
    plt.title(title)
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

plot_bdm_results(tumor_bdm_results, "Top 10 Tumor BDM Perturbations", os.path.join(tumor_dir, "top_10_tumor_bdm2.png"))
plot_bdm_results(normal_bdm_results, "Top 10 Normal BDM Perturbations", os.path.join(normal_dir, "top_10_normal_bdm2.png"))

print("BDM perturbation analysis completed and results saved.")


BDM perturbation analysis completed and results saved.


In [5]:

# Define directories
base_dir = "C:/Users/uabic/Desktop/pAML_1/pAML_1"
normal_dir = os.path.join(base_dir, "Normal")
tumor_dir = os.path.join(base_dir, "Tumor")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os

# Load adjacency matrices
tumor_adj_matrix = pd.read_csv(os.path.join(tumor_dir, "tumor_adjacency_matrix2.csv"), index_col=0)
normal_adj_matrix = pd.read_csv(os.path.join(normal_dir, "normal_adjacency_matrix2.csv"), index_col=0)

# Function to plot the network
def plot_network(adj_matrix, title, save_path):
    G = nx.from_pandas_adjacency(adj_matrix)
    pos = nx.spring_layout(G)
    plt.figure(figsize=(24, 16))
    nx.draw(G, pos, with_labels=False, node_color='skyblue', edge_color='pink', node_size=500)
    
    # Move the labels to avoid overlap
    pos_labels = {node: (x, y + 0.1) for (node, (x, y)) in pos.items()}
    nx.draw_networkx_labels(G, pos_labels, font_size=24, font_color='black')
    
    plt.title(title, fontsize=24)
    plt.tight_layout()
    plt.savefig(save_path, format='jpeg', pil_kwargs={'quality': 95})
    plt.close()

# Plot the networks
plot_network(tumor_adj_matrix.abs(), 'Tumor Network', os.path.join(tumor_dir, 'tumor_network.jpeg'))
plot_network(normal_adj_matrix.abs(), 'Normal Network', os.path.join(normal_dir, 'normal_network.jpeg'))

# Function to compute centralities and save top 10 as CSV
def compute_and_save_centralities(adj_matrix, folder, prefix):
    G = nx.from_pandas_adjacency(adj_matrix)
    centralities = {
        'betweenness': nx.betweenness_centrality(G),
        'closeness': nx.closeness_centrality(G),
        'eigenvector': nx.eigenvector_centrality(G, max_iter=1000)
    }
    for centrality_name, centrality_values in centralities.items():
        centrality_df = pd.DataFrame.from_dict(centrality_values, orient='index', columns=[centrality_name])
        top_10_centrality_df = centrality_df.nlargest(10, centrality_name)
        top_10_centrality_df.to_csv(os.path.join(folder, f"{prefix}_{centrality_name}_top_10.csv"))

# Compute and save centralities for tumor and normal networks
compute_and_save_centralities(tumor_adj_matrix.abs(), tumor_dir, "tumor")
compute_and_save_centralities(normal_adj_matrix.abs(), normal_dir, "normal")

# Function to compute transitivity and diameter, then save as CSV
def compute_and_save_transitivity_diameter(adj_matrix, folder, prefix):
    G = nx.from_pandas_adjacency(adj_matrix)
    transitivity = nx.transitivity(G)
    try:
        diameter = nx.diameter(G)
    except nx.NetworkXError:  # Handle disconnected graphs
        diameter = float('inf')
    metrics_df = pd.DataFrame({
        'Metric': ['Transitivity', 'Diameter'],
        'Value': [transitivity, diameter]
    })
    metrics_df.to_csv(os.path.join(folder, f"{prefix}_metrics.csv"), index=False)

# Compute and save transitivity and diameter for tumor and normal networks
compute_and_save_transitivity_diameter(tumor_adj_matrix.abs(), tumor_dir, "tumor")
compute_and_save_transitivity_diameter(normal_adj_matrix.abs(), normal_dir, "normal")


In [ ]:

#DEG Analysis with BDM

# Import necessary libraries
import os
import pandas as pd
import numpy as np
from pybdm import BDM
from pybdm import PerturbationExperiment
from scipy.stats import spearmanr  # Add this import statement
import matplotlib.pyplot as plt

# Define directories
base_dir = os.path.expanduser("~/Desktop/DEG")
sub_dirs = ['AML', 'Breast', 'Pros']
data_files = {
    'AML': ('DEGAML.csv', 'exprAML.csv'),
    'Breast': ('DEGBreast.csv', 'exprBreast.csv'),
    'Pros': ('DEGPros.csv', 'exprPros.csv')
}

sample_info = {
    'AML': {
        "GSM100895.CEL.gz": "N", "GSM100896.CEL.gz": "N", "GSM100897.CEL.gz": "N", "GSM100898.CEL.gz": "N",
        "GSM100901.CEL.gz": "N", "GSM187698.CEL.gz": "T", "GSM187701.CEL.gz": "T", "GSM187704.CEL.gz": "T",
        "GSM187707.CEL.gz": "T", "GSM187710.CEL.gz": "T", "GSM187713.CEL.gz": "T", "GSM187716.CEL.gz": "T"
    },
    'Breast': {
        "GSM85473.CEL": "T", "GSM85474.CEL": "T", "GSM85475.CEL": "T", "GSM85476.CEL": "T",
        "GSM85477.CEL": "T", "GSM85478.CEL": "T", "GSM85479.CEL": "T", "GSM85480.CEL": "T",
        "GSM85481.CEL": "T", "GSM85482.CEL": "T", "GSM85483.CEL": "T", "GSM85484.CEL": "T",
        "GSM85485.CEL": "T", "GSM85486.CEL": "T", "GSM85487.CEL": "T", "GSM85488.CEL": "T",
        "GSM85489.CEL": "T", "GSM85490.CEL": "T", "GSM85491.CEL": "T", "GSM85492.CEL": "T",
        "GSM85493.CEL": "T", "GSM85494.CEL": "T", "GSM85495.CEL": "T", "GSM85496.CEL": "T",
        "GSM85497.CEL": "T", "GSM85498.CEL": "T", "GSM85499.CEL": "T", "GSM85500.CEL": "T",
        "GSM85501.CEL": "T", "GSM85502.CEL": "T", "GSM85503.CEL": "T", "GSM85504.CEL": "T",
        "GSM85505.CEL": "T", "GSM85506.CEL": "T", "GSM85507.CEL": "T", "GSM85508.CEL": "T",
        "GSM85509.CEL": "T", "GSM85510.CEL": "T", "GSM85511.CEL": "T", "GSM85512.CEL": "T",
        "GSM85513.CEL": "N", "GSM85514.CEL": "N", "GSM85515.CEL": "N", "GSM85516.CEL": "N",
        "GSM85517.CEL": "N", "GSM85518.CEL": "N", "GSM85519.CEL": "N"
    },
    'Pros': {
        "GSM74875.CEL.gz": "N", "GSM74876.CEL.gz": "N", "GSM74877.CEL.gz": "N", "GSM74878.CEL.gz": "N",
        "GSM74879.CEL.gz": "N", "GSM74880.CEL.gz": "N", "GSM74881.CEL.gz": "N", "GSM74882.CEL.gz": "T",
        "GSM74883.CEL.gz": "T", "GSM74884.CEL.gz": "T", "GSM74885.CEL.gz": "T", "GSM74886.CEL.gz": "T",
        "GSM74887.CEL.gz": "T", "GSM74888.CEL.gz": "T", "GSM74889.CEL.gz": "T", "GSM74890.CEL.gz": "T",
        "GSM74891.CEL.gz": "T", "GSM74892.CEL.gz": "T", "GSM74893.CEL.gz": "T"
    }
}

# Create subfolders for results
for sub_dir in sub_dirs:
    os.makedirs(os.path.join(base_dir, sub_dir), exist_ok=True)

def filter_degs(deg_file):
    deg_df = pd.read_csv(deg_file)
    filtered_degs = deg_df[(deg_df['logFC'] >= 1.00) | (deg_df['logFC'] <= -1.00)]
    return filtered_degs.drop_duplicates(subset='Gene')

def separate_samples(expr_file, sample_info):
    data = pd.read_csv(expr_file)
    sample_info_series = pd.Series(sample_info)
    tumor_samples = sample_info_series[sample_info_series == "T"].index
    normal_samples = sample_info_series[sample_info_series == "N"].index
    tumor_data = data[['Gene'] + tumor_samples.tolist()]
    normal_data = data[['Gene'] + normal_samples.tolist()]
    return tumor_data.set_index('Gene'), normal_data.set_index('Gene')

def create_adjacency_matrix(data):
    gene_names = data.index
    expression_data = data.values
    corr_matrix = spearmanr(expression_data, axis=1).correlation
    np.fill_diagonal(corr_matrix, 0)  # Remove self-correlations
    return pd.DataFrame(corr_matrix, index=gene_names, columns=gene_names)

def binarize_matrix(matrix, threshold=0.5):
    binary_matrix = (matrix > threshold).astype(int)
    return binary_matrix

def bdm_perturbation_analysis(adj_matrix):
    bdm = BDM(ndim=2)
    binary_matrix = binarize_matrix(adj_matrix.values)
    perturbation = PerturbationExperiment(bdm, binary_matrix, metric='bdm')
    delta_bdm = perturbation.run()
    
    # Ensure the shape matches the original matrix
    reshaped_delta_bdm = np.reshape(delta_bdm, adj_matrix.shape)
    
    return pd.DataFrame(reshaped_delta_bdm, index=adj_matrix.index, columns=adj_matrix.columns)

def save_bdm_results(gene_names, bdm_results, file_path):
    result_df = pd.DataFrame({'Gene': gene_names, 'BDM': bdm_results})
    result_df.to_csv(file_path, index=False)

def plot_top_10_bdm(file_path, title, save_path):
    bdm_results = pd.read_csv(file_path)
    top_10 = bdm_results.nlargest(10, 'BDM')
    plt.figure(figsize=(10, 6))
    plt.bar(top_10['Gene'], top_10['BDM'])
    plt.xlabel('Gene')
    plt.ylabel('BDM')
    plt.title(title)
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# Process each dataset sequentially
for cancer_type, (deg_file, expr_file) in data_files.items():
    print(f"Processing {cancer_type} dataset...")
    
    deg_file_path = os.path.join(base_dir, deg_file)
    expr_file_path = os.path.join(base_dir, expr_file)
    filtered_degs = filter_degs(deg_file_path)
    
    tumor_data, normal_data = separate_samples(expr_file_path, sample_info[cancer_type])
    
    # Ensure genes are present in both DEG list and expression data
    common_genes = filtered_degs['Gene'][filtered_degs['Gene'].isin(tumor_data.index)]
    tumor_data = tumor_data.loc[common_genes]
    normal_data = normal_data.loc[common_genes]
    
    # Create adjacency matrices
    tumor_adj_matrix = create_adjacency_matrix(tumor_data)
    normal_adj_matrix = create_adjacency_matrix(normal_data)
    
    
tumor_bdm_results = bdm_perturbation_analysis(tumor_adj_matrix)
tumor_bdm_results.to_csv(os.path.join(base_dir, cancer_type, f"{cancer_type}_tumor_bdm_results.csv"))
plot_top_10_bdm(os.path.join(base_dir, cancer_type, f"{cancer_type}_tumor_bdm_results.csv"), f"Top 10 {cancer_type} Tumor BDM", os.path.join(base_dir, cancer_type, f"top_10_{cancer_type}_tumor_bdm.jpeg"))

normal_bdm_results = bdm_perturbation_analysis(normal_adj_matrix)
normal_bdm_results.to_csv(os.path.join(base_dir, cancer_type, f"{cancer_type}_normal_bdm_results.csv"))
plot_top_10_bdm(os.path.join(base_dir, cancer_type, f"{cancer_type}_normal_bdm_results.csv"), f"Top 10 {cancer_type} Normal BDM", os.path.join(base_dir, cancer_type, f"top_10_{cancer_type}_normal_bdm.jpeg"))


print(f"Finished processing {cancer_type} dataset.")

print("BDM analysis completed for all datasets.")